In [1]:
import os
import cv2
import numpy as np
from imutils import paths
import pickle

In [2]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

In [3]:
from keras.applications import ResNet50

from keras.layers import Input
from keras.layers.pooling import AveragePooling2D
from keras.layers.core import Flatten
from keras.layers.core import Dense

from keras.layers.core import Dropout

from keras.models import Model

from keras.optimizers import SGD

In [4]:
dataPath = r"./Data"
outputModel = r"./Model/ClassificationModel"
ouputLabelBinarizer = r"./Model/ClassificationBinarizer"

In [5]:
# Reproducibility

epoch = 25

In [6]:
Sports_Labels = set(['boxing','swimming','wwe','football','basketball','table_tennis','hockey'])
print ("Images are being loaded ....")
pathToImages = list(paths.list_images(dataPath)) 
data = []
labels = []

# PreProcessing of Data

for images in pathToImages:
    label = images.split(os.path.sep)[-2]
    if label not in Sports_Labels:
        continue
    image = cv2.imread(images)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224,244))
    data.append(image)
    labels.append(label)

Images are being loaded ....


In [7]:
# Coverting data and labels to numpy arrays for model training
data = np.array(data)
labels = np.array(labels)

# Hot Encoded Values as 0,1,2,3,4,5,6
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
print(labels)

[[1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]]


In [8]:
# train test data split
(X_train, X_test, Y_train, Y_test) = train_test_split(data,labels,test_size=0.25,stratify=labels,random_state=42)

In [9]:
# Augmentation
trainingAugmentation = ImageDataGenerator(
    rotation_range=30,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Validation Augmentation
validationAugmentation = ImageDataGenerator()

mean = np.array([123.68, 116.779, 103,939] , dtype="float32")
trainingAugmentation.mean = mean
validationAugmentation.mean = mean

In [10]:
baseModel = ResNet50(weights="imagenet", include_top=False, input_tensor=Input(shape=(224,244,3)))

headModel = baseModel.output
haedModel = AveragePooling2D(pool_size=(7,7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(512,activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(len(lb.classes_), activation="softmax")(headModel)
model = Model(inputs=baseModel.input , outputs=headModel)

for baseModelLayers in baseModel.layers:
    baseModelLayers.trainable = False

In [11]:
opt = SGD(lr=0.0001, momentum=0.9, decay=1e-4/epoch)

In [15]:
model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=["accuracy"])

In [16]:
History = model.fit_generator(
    trainingAugmentation.flow(X_train,Y_train, batch_size=32),
    steps_per_epoch=len(X_train) // 32,
    validation_data=validationAugmentation.flow(X_test,Y_test),
    validation_steps=len(X_test) // 32,
    epochs=epoch
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/25
107/107 [==============================] - 561s 5s/step - loss: 1.5909 - accuracy: 0.5712 - val_loss: 0.4919 - val_accuracy: 0.8232
Epoch 2/25
107/107 [==============================] - 584s 5s/step - loss: 0.7541 - accuracy: 0.7306 - val_loss: 0.4004 - val_accuracy: 0.8598
Epoch 3/25
107/107 [==============================] - 558s 5s/step - loss: 0.6387 - accuracy: 0.7789 - val_loss: 0.3629 - val_accuracy: 0.8687
Epoch 4/25
107/107 [==============================] - 588s 5s/step - loss: 0.5661 - accuracy: 0.8061 - val_loss: 0.3260 - val_accuracy: 0.8866
Epoch 5/25
107/107 [==============================] - 572s 5s/step - loss: 0.5099 - accuracy: 0.8140 - val_loss: 0.2996 - val_accuracy: 0.9009
Epoch 6/25
107/107 [==============================] - 532s 5s/step - loss: 0.4861 - accuracy: 0.8280 - val_loss: 0.2857 - val_accuracy: 0.9009
Epoch 7/25
107/107 [==============================] - 528s 5s/step

In [17]:
model.save(outputModel)
lblBinarizer = open(r"./Model/ClassificationBinarizer/ClassificationBinarizer.pickle",'wb')
lblBinarizer.write(pickle.dumps(lb))
lblBinarizer.close()

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ./Model/ClassificationModel\assets
